<a href="https://colab.research.google.com/github/saeedsrm/NLP/blob/main/NLP%20check%20the%20comment%20reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive to load model of Word2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from gensim.models import KeyedVectors

# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Computational Intelligence/GoogleNews-vectors-negative300.bin', binary=True)

# Access vectors for specific words with a keyed lookup:
vector = model['easy']
# see the shape of the vector (300,)
vector.shape

# Processing sentences is not as simple as with Spacy:
vectors = [model[x] for x in "This is some text I am processing with Spacy".split(' ')]

get Shape

In [ ]:
vector = model['love']
# see the shape of the vector (300,)
print(vector.shape)

(300,)


some tests

In [ ]:
vectors = [model[x] for x in "This is some text I am processing with Spacy".split(' ')]
for i in range(len (vectors)):
  vectors[i].shape 

model.similarity('cat','dog')
# model.most_similar('dog')

0.76094574

Load Data

The first step

get train data

In [ ]:
import csv
import re
x_train=[]
y_train=[]
with open("/content/drive/MyDrive/Computational Intelligence/dataset/train.csv", 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row in csvreader:
      tmp=row[1]
      specialChars = '0123456879[\!"#$%&\*+,-./:;<=>?@^_`()|~=]\\n' 
      for specialChar in specialChars:
        tmp = tmp.replace(specialChar, '')
      x_train.append(tmp)
      y_train.append(row[2])

get test data

In [ ]:
x_test=[]
y_test=[]
with open("/content/drive/MyDrive/Computational Intelligence/dataset/test.csv", 'r') as file:
    csvreader = csv.reader(file)
    header = next(csvreader)
    for row in csvreader:
      tmp=row[1]
      specialChars = '0123456879[\!"#$%&\*+,-./:;<=>?@^_`()|~=]\\n' 
      for specialChar in specialChars:
        tmp = tmp.replace(specialChar, '')
      x_test.append(tmp)
      y_test.append(row[2])

In [ ]:
import pandas as pd
# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

# set up to view all the info of the columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import numpy as np
# train data
x_train=np.array(x_train)
y_train=np.array(y_train)
x_train_pd=pd.DataFrame(x_train)
y_train_pd=pd.DataFrame(y_train)

# test data
x_test=np.array(x_test)
y_test=np.array(y_test)
x_test_pd=pd.DataFrame(x_test)
y_test_pd=pd.DataFrame(y_test)

x_train_pd.head()
# y_train_pd.head()

0
0  A few thigs You might have egative frequecy de...
1  Is it so hard to believe that there exist part...
2                                     There are bees
3  I'm a medicatio techicia Ad that's alot of dru...
4                      Cesium is such a pretty metal

The second step

In [ ]:
x_train_pd.shape

(8695, 1)

In [ ]:
def basic_info(df):
    print("This dataset has ", df.shape[1], " columns and ", df.shape[0], " rows.")
    print("This dataset has ", df[df.duplicated()].shape[0], " duplicated rows.")
    print(" ")
    print("Descriptive statistics of the numeric features in the dataset: ")
    print(" ")
    print(df.describe())
    print(" ")
    print("Information about this dataset: ")
    print(" ")
    print(df.info())

In [ ]:
basic_info(x_train_pd)
basic_info(y_train_pd)

This dataset has  1  columns and  8695  rows.
This dataset has  776  duplicated rows.
 
Descriptive statistics of the numeric features in the dataset: 
 
              0
count      8695
unique     7919
top     removed
freq        114
 
Information about this dataset: 
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8695 entries, 0 to 8694
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8695 non-null   object
dtypes: object(1)
memory usage: 68.1+ KB
None
This dataset has  1  columns and  8695  rows.
This dataset has  8692  duplicated rows.
 
Descriptive statistics of the numeric features in the dataset: 
 
              0
count      8695
unique        3
top     Biology
freq       3591
 
Information about this dataset: 
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8695 entries, 0 to 8694
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       8695 n

**BOW and TF_IDF**

create vector for sentece with BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
 
CountVec = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')
#transform
Count_data = CountVec.fit_transform(x_train_pd[0])
 
#create dataframe
x_train_sec2vec=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())


#transform
Count_data = CountVec.fit_transform(x_test_pd[0])
 
#create dataframe
x_test_sec2vec=pd.DataFrame(Count_data.toarray(),columns=CountVec.get_feature_names())


# x_train_sec2vec.head()

**add word2vec**

In [ ]:
not_word=[]
word_vectors=model.wv

for i in range (len(x_train_sec2vec.columns)):
  if x_train_sec2vec.columns[i] not in word_vectors.vocab:
    not_word.append(x_train_sec2vec.columns[i])
print(x_train_sec2vec.shape)
x_train_sec2vec=x_train_sec2vec.drop(columns=not_word)
print(x_train_sec2vec.shape)

(8695, 18583)
(8695, 8530)


In [ ]:
distanse=[]
for i in range (len(x_train_sec2vec.columns)):
  tmp=[]
  for j in range (len(x_train_sec2vec.columns)):
    tmp.append(0)
  distanse.append(tmp)

for i in range (len(x_train_sec2vec.columns)):
  for j in range (i+1,len(x_train_sec2vec.columns)):
    score=model.similarity(x_train_sec2vec.columns[j],x_train_sec2vec.columns[i])
    distanse[i][j]=score
    distanse[j][i]=score
print (distanse)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
x_train_sec2vec_tmp=x_train_sec2vec.to_numpy()
for k in range(len(x_train_sec2vec_tmp)):
  for i in range(len(x_train_sec2vec_tmp[k])):
    if x_train_sec2vec_tmp[k][i]==1:
      for j in range (len(x_train_sec2vec.columns)):
        score=distanse[j][i]
        if score>0.5 and score>x_train_sec2vec_tmp[k][j]:
          x_train_sec2vec_tmp[k][j]=score


x_train_sec2vec=pd.DataFrame(x_train_sec2vec_tmp,columns=x_train_sec2vec.columns)
x_train_sec2vec.head()

0     1     2     3     4     5     6     7     8     9     10    11    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   12    13    14    15    16    17    18    19    20    21    22    23    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   24    25    26    27    28    29    30    31    32    33    34    35    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   36    37    38    39    40    41    42    43    44    45    46    47    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   48    49    50    51    52    53    54    55    56    57    58    59    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   60    61    62    63    64    65    66    67    68    69    70    71    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   72    73    74    75    76    77    78    79    80    81    82    83    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   84    85    86    87    88    89    90    91    92    93    94    95    \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   96    97    98    99    100   101   102   103   104   105   106   107   \
0     0     0     0     0     0     0     0     0     0     0     6     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0 

**clustering vector of word**

Clustering

In [ ]:
x_train_sec2vec.columns

In [ ]:
import  numpy as np
distances = []
words = x_train_sec2vec.columns
for i in range(len(words)):
  j = i
  distances.append([])
  while j < len(words):
    d = np.linalg.norm(model[words[j]] - model[words[i]])
    distances[i].append(d)
    j += 1

In [ ]:
def retrieve_distance(distances, i, j):
    if i >= j:
        d = distances[j][i-j]
    else:
        d = distances[i][j-i]
    return d

def find_friendliest(distances):
    min = 10000
    index = -1
    for i in range(len(distances)):
        sum = 0
        for j in range(len(distances)):
            if i == j:
                continue
            d = retrieve_distance(distances, i, j)
            sum += d
        average = sum/len(distances)
        if average < min:
            min = average
            index = i
    return index
    
         
def find_next_max_min(distances, selected_items, not_selected_items):
    max = -1
    index = -1*/ x
    for i in not_selected_items:
        min = 100000
        for j in selected_items:
            if i == j:
                continue
            d = retrieve_distance(distances, i, j)
            if d < min:
                min = d

        if min > max :
            max = min
            index = i
    return index

In [ ]:
friendliest = find_friendliest(distances) # I1 => arg(min(1/m * sigma(D[i,j])))
not_selected_items = [i for i in range(len(distances))]
not_selected_items.remove(friendliest) # K1
selected_items = [friendliest] # S1

In [ ]:
C = 100
n = 1
while n < C:
    # arg (max(min(D[IkIj])))
    next = find_next_max_min(distances, selected_items, not_selected_items)
    selected_items.append(next) # K(n+1)
    not_selected_items.remove(next) # S(n+1)
    n += 1
clusters = []
for i in selected_items:
  clusters.append([words[i]])

for i, word in enumerate(words):
  min = 10000
  min_j = -1
  for j, selected_item in enumerate(selected_items):
    d = retrieve_distance(distances, i, j)
    if d < min:
      min = d
      min_j = j
  clusters[min_j].append(words[i])

In [ ]:
def get_neighbours(word):
  for cluster in clusters:
    for item in cluster:
      if (item == word):
        return cluster

In [ ]:
print(get_neighbours('like'))




In [ ]:
x_train_sec2vec_tmp=x_train_sec2vec.to_numpy()
for k in range(len(x_train_sec2vec_tmp)):
  words=[]
  for i in range(len(x_train_sec2vec_tmp[k])):
    if x_train_sec2vec_tmp[k][i]==1:
      words=words+get_neighbours(x_train_sec2vec.columns[i])
  for j in range (len(x_train_sec2vec.columns)):
    if x_train_sec2vec.columns[j] in words:
      x_train_sec2vec_tmp[k][j]=0.5


x_train_sec2vec=pd.DataFrame(x_train_sec2vec_tmp,columns=x_train_sec2vec.columns)
x_train_sec2vec.head()

create vector for sentece with TI_IDF

In [ ]:
#without smooth IDF
print("Without Smoothing:")
#define tf-idf
tf_idf_vec = TfidfVectorizer(use_idf=True, 
                        smooth_idf=False,  
                        ngram_range=(1,1),stop_words='english') # to use only  bigrams ngram_range=(2,2)
#transform
tf_idf_data = tf_idf_vec.fit_transform(x_train_pd[0])
 
#create dataframe
tf_idf_dataframe=pd.DataFrame(tf_idf_data.toarray(),columns=tf_idf_vec.get_feature_names())
print(tf_idf_dataframe.shape)
# print("\n")
 
# #with smooth
tf_idf_vec_smooth = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(1,1),stop_words='english')
 
 
tf_idf_data_smooth = tf_idf_vec_smooth.fit_transform(x_train_pd[0])
 
print("With Smoothing:")
tf_idf_dataframe_smooth=pd.DataFrame(tf_idf_data_smooth.toarray(),columns=tf_idf_vec_smooth.get_feature_names())
print(tf_idf_dataframe_smooth.head())


Streaming output truncated to the last 5000 lines.
3       0.0        0.0        0.0        0.0         0.0                0.0   
4       0.0        0.0        0.0        0.0         0.0                0.0   

   regulator  regulatory  regurgitate  regurgitated  regurgitatig  rehabber  \
0        0.0         0.0          0.0           0.0           0.0       0.0   
1        0.0         0.0          0.0           0.0           0.0       0.0   
2        0.0         0.0          0.0           0.0           0.0       0.0   
3        0.0         0.0          0.0           0.0           0.0       0.0   
4        0.0         0.0          0.0           0.0           0.0       0.0   

   reideer  reideeredit  reideers  reifected  reifectio  reiforces  \
0      0.0          0.0       0.0        0.0        0.0        0.0   
1      0.0          0.0       0.0        0.0        0.0        0.0   
2      0.0          0.0       0.0        0.0        0.0        0.0   
3      0.0          0.0       0.0  

**classification with support vector machine (SVM)**

kernels:



1.   linear: In the case of a simple SVM we simply set this parameter as "linear" since simple SVMs can only classify linearly separable data
3.   poly: In the case of polynomial kernel, you also have to pass a value for the degree parameter of the SVC class.

4.   rbf: To use Gaussian kernel, you have to specify 'rbf' as value for the Kernel parameter of the SVC class.

5.   sigmoid


In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf')
svclassifier.fit(x_train_sec2vec, y_train_pd[0])


SVC()

perdict lable of test data and get report

In [ ]:
y_pred = svclassifier.predict(x_test_sec2vec)
from sklearn.metrics import classification_report, confusion_matrix
print('this is confusion_matrix for y_test and y_perd\n')
print(confusion_matrix(y_test,y_pred))
print ('\n\nthis is classification_report\n')
print(classification_report(y_test,y_pred))
print (y_test.unique())